In [1]:
from dotenv import load_dotenv
import os
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy
from datetime import datetime

In [2]:
load_dotenv()


False

In [ ]:
class TelegramBot:
    def __init__(self):
        self.TOKEN = os.getenv("API_KEY")
        self.url = f"https://api.telegram.org/bot{self.TOKEN}/"

    def start(self):
        update_id = None
        while True:
            update = self.get_message(update_id)
            messages = update.get('result', [])
            if messages:
                for message in messages:
                    try:
                        update_id = message['update_id']
                        chat_id = message['message']['chat']['id']  # Corrigido: use 'chat' em vez de 'from'
                        message_text = message['message']['text']
                        answer_bot = self.create_answer(message_text)
                        self.send_answer(chat_id, answer_bot)
                    except Exception as e:
                        print(f"Erro ao processar mensagem: {e}")

    def get_message(self, update_id):
        link_request = f"{self.url}getUpdates?timeout=1000"
        if update_id:
            link_request += f"&offset={update_id + 1}"  # Corrigido: adicionado '&'
        result = requests.get(link_request)
        return result.json()

    def create_answer(self, message_text):
        dataframe = transform_data(self.driveBot.get_data())
        message_text = message_text.lower()
        if message_text in ["/start", "ola", "eae", "menu", "oi", "oie"]:
            return "Ola, tudo bem? Seja bem vindo ao Bot do RH da Empresa RDS. Selecione o que deseja:" + "\n" + "1 - NPS interno mensal médio por setor" + "\n" + "2 - NPS interno mensal médio por contratação" + "\n" + "3 - Distribuição do NPS interno" + "\n", 0
        elif message_text == '1':
            return barv_npsmean_by(dataframe, "Setor"), 1
        elif message_text == '2':
            return barv_npsmean_by(dataframe, "Tipo de Contratação"), 1
        elif message_text == '3':
            return hist_nps(dataframe), 1
        else:
            return "Comando não encontrado, tente novamente. Selecione o que deseja:" + "\n" + "1 - NPS interno mensal médio por setor" + "\n" + "2 - NPS interno mensal médio por contratação" + "\n" + "3- Distribuição do NPS interno" + "\n", 0
    

    def send_answer(self, chat_id, answer, figure_boolean):
        if figure_boolean == 0:
            link_to_send = f"{self.url}sendMessage?chat_id={chat_id}&text={answer}"
            requests.get(link_to_send)
            return
        else:
            answer.seek(0)
            requests.post(f"{self.url}sendPhoto?chat_id={chat_id}", files = dict(photo=answer))
            answer.close()
            return

# Crie uma instância da classe TelegramBot
bot = TelegramBot()

# Chame o método 'start' na instância do bot
bot.start()


In [2]:
dados = pd.read_csv("data/healthcare_dataset.csv")
dados.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.983364,146,Elective,2022-12-01,Aspirin,Inconclusive
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,404,Emergency,2023-06-15,Lipitor,Normal
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.896997,292,Emergency,2019-02-08,Lipitor,Normal
3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,23303.322092,480,Urgent,2020-05-03,Penicillin,Abnormal
4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,Dustin Griffin,"Jones, Brown and Murray",UnitedHealthcare,18086.344184,477,Urgent,2021-08-02,Paracetamol,Normal


In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                10000 non-null  object 
 1   Age                 10000 non-null  int64  
 2   Gender              10000 non-null  object 
 3   Blood Type          10000 non-null  object 
 4   Medical Condition   10000 non-null  object 
 5   Date of Admission   10000 non-null  object 
 6   Doctor              10000 non-null  object 
 7   Hospital            10000 non-null  object 
 8   Insurance Provider  10000 non-null  object 
 9   Billing Amount      10000 non-null  float64
 10  Room Number         10000 non-null  int64  
 11  Admission Type      10000 non-null  object 
 12  Discharge Date      10000 non-null  object 
 13  Medication          10000 non-null  object 
 14  Test Results        10000 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 1.1+

In [5]:
dados['Discharge Date'] = pd.to_datetime(dados['Discharge Date'])
dados['Discharge Date'] = dados['Discharge Date'].dt.date
dados['Discharge Date'] = pd.to_datetime(dados['Discharge Date']).dt.normalize()


In [6]:
dados['Date of Admission'] = pd.to_datetime(dados['Date of Admission'])
dados['Date of Admission'] = dados['Date of Admission'].dt.date
dados['Date of Admission'] = pd.to_datetime(dados['Date of Admission']).dt.normalize()

In [7]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Name                10000 non-null  object        
 1   Age                 10000 non-null  int64         
 2   Gender              10000 non-null  object        
 3   Blood Type          10000 non-null  object        
 4   Medical Condition   10000 non-null  object        
 5   Date of Admission   10000 non-null  datetime64[ns]
 6   Doctor              10000 non-null  object        
 7   Hospital            10000 non-null  object        
 8   Insurance Provider  10000 non-null  object        
 9   Billing Amount      10000 non-null  float64       
 10  Room Number         10000 non-null  int64         
 11  Admission Type      10000 non-null  object        
 12  Discharge Date      10000 non-null  datetime64[ns]
 13  Medication          10000 non-null  object     

In [7]:
dados["Date of Admission"].min()


Timestamp('2018-10-30 00:00:00')

In [8]:
dados["Discharge Date"].min()

Timestamp('2018-11-01 00:00:00')

In [9]:
dados.head(3)

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.983364,146,Elective,2022-12-01,Aspirin,Inconclusive
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,404,Emergency,2023-06-15,Lipitor,Normal
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.896997,292,Emergency,2019-02-08,Lipitor,Normal


In [33]:
def media_por_sexo(dados):
    media_por_sexo = dados.groupby('Gender')['Billing Amount'].mean()
    return media_por_sexo

media_por_sexo = media_por_sexo(dados)


In [27]:
counts = dados["Medication"].value_counts()

In [28]:
print(counts)

Penicillin     2079
Lipitor        2015
Ibuprofen      1976
Aspirin        1968
Paracetamol    1962
Name: Medication, dtype: int64


In [30]:
def media_por_sangue(dados):
    media_por_sangue = dados.groupby('Blood Type')['Billing Amount'].mean()
    return media_por_sangue

media_por_sangue = media_por_sangue(dados)

In [31]:
def media_por_medicacao(dados):
    media_por_medicacao = dados.groupby('Medication')['Billing Amount'].mean()
    return media_por_medicacao 

media_por_medicacao = media_por_medicacao(dados)

In [32]:
print(media_por_medicacao)

Medication
Aspirin        25795.641295
Ibuprofen      25439.097865
Lipitor        26126.263179
Paracetamol    25145.410639
Penicillin     25086.518686
Name: Billing Amount, dtype: float64


In [35]:
import os
import requests

class TelegramBot:
    def __init__(self):
        self.TOKEN = os.getenv("API_KEY")
        self.url = f"https://api.telegram.org/bot{self.TOKEN}/"

    def start(self):
        update_id = None
        while True:
            update = self.get_message(update_id)
            messages = update.get('result', [])
            if messages:
                for message in messages:
                    try:
                        update_id = message['update_id']
                        chat_id = message['message']['chat']['id']  # Corrigido: use 'chat' em vez de 'from'
                        message_text = message['message']['text']
                        answer_bot, status = self.create_answer(message_text)
                        self.send_answer(chat_id, answer_bot)
                    except Exception as e:
                        print(f"Erro ao processar mensagem: {e}")

    def get_message(self, update_id):
        link_request = f"{self.url}getUpdates?timeout=1000"
        if update_id:
            link_request += f"&offset={update_id + 1}"  # Corrigido: adicionado '&'
        result = requests.get(link_request)
        return result.json()

    def create_answer(self, message_text):
        # Substitua com a lógica adequada para criar respostas
        if message_text in ["/start", "ola", "eae", "menu", "oi", "oie"]:
            return ("Ola, tudo bem? Seja bem-vindo ao Bot do RH da Empresa RDS. Selecione o que deseja:\n"
                    "1 - NPS interno mensal médio por setor\n"
                    "2 - NPS interno mensal médio por contratação\n"
                    "3 - Distribuição do NPS interno\n", 0)
        elif message_text == '1':
            # Substitua com a lógica adequada para calcular a média_por_sexo
            media_por_sexo = "Lógica para calcular a média_por_sexo"
            return (media_por_sexo, 1)
        elif message_text == '2':
            # Substitua com a lógica adequada para calcular a media_por_sangue
            media_por_sangue = "Lógica para calcular a media_por_sangue"
            return (media_por_sangue, 1)
        elif message_text == '3':
            # Substitua com a lógica adequada para calcular a media_por_medicacao
            media_por_medicacao = "Lógica para calcular a media_por_medicacao"
            return (media_por_medicacao, 1)
        else:
            return ("Comando não encontrado, tente novamente. Selecione o que deseja:\n"
                    "1 - NPS interno mensal médio por setor\n"
                    "2 - NPS interno mensal médio por contratação\n"
                    "3 - Distribuição do NPS interno\n", 0)

    def send_answer(self, chat_id, answer):
        # Adicione a lógica para enviar a resposta para o chat_id
        pass

# Crie uma instância da classe TelegramBot
bot = TelegramBot()

# Chame o método 'start' na instância do bot
bot.start()


KeyboardInterrupt: 